In [4]:
import os
import gc
import torch

import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging, TextStreamer

from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
import bitsandbytes as bnb

import wandb

In [2]:
inference_model_id = "ogbrandt/bfloat-16-merge"

In [18]:
!nvidia-smi

Wed Jan 31 03:58:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.154.05             Driver Version: 535.154.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A10G                    Off | 00000000:00:1E.0 Off |                    0 |
|  0%   25C    P0              57W / 300W |   6584MiB / 23028MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch 

model_name = "ogbrandt/mist7b-pjf-ft-dpo-v1"

model = AutoModelForCausalLM.from_pretrained(
  model_name, load_in_4bit=True, 
    device_map= "auto")
# model = AutoModelForCausalLM.from_pretrained(
#     model_name, low_cpu_mem_usage=True,
#     return_dict=True,torch_dtype=torch.bfloat16,
#     device_map= "auto")

TypeError: MistralForCausalLM.__init__() got an unexpected keyword argument 'padding_side'

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, padding_size="left")
tokenizer.pad_token = tokenizer.eos_token

In [24]:
# https://huggingface.co/docs/transformers/llm_tutorial
model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
generated_ids = model.generate(**model_inputs, max_new_tokens=200, do_sample=True, temperature=0.3, stopping_criteria=stopping_criteria)

# decoding strats: https://huggingface.co/blog/how-to-generate
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"<|im_start|>system\nYou are a helpful athletic coach mirroring Paul J Fabritz called PaulGPT<|im_end|>\n<|im_start|>user\nHello, how are you?<|im_end|>\n<|im_start|>assistant\nI'm doing great. How can I help you today?<|im_end|>\n<|im_start|>user\nhow to dunk?<|im_end|>\n<|im_start|>assistant\nTo dunk, start by jumping as high as you can. Then, work on increasing your vertical jump by incorporating plyometric exercises like bounding and bounding with a mini-tramp. As you improve, gradually add more intensity to your jumps, such as by using a rebounder or by increasing the height of the box you're jumping from. Also, focus on improving your overall athleticism and explosiveness, as this will have a significant impact on your ability to dunk. Additionally, work on your approach and technique to ensure a smooth and effective transition from running to jumping. With dedication and practice, you can significantly improve your vertical jump and eventually master the art of dunking<|im_end|>

In [21]:
chat = [
  {"role": "system", "content": "You are a helpful athletic coach mirroring Paul J Fabritz called PaulGPT"},
  {"role": "user", "content": "Hello, how are you?"},
  {"role": "assistant", "content": "I'm doing great. How can I help you today?"},
  {"role": "user", "content": "how to dunk?"},
]

tokenizer.chat_template = "{% if not add_generation_prompt is defined %}{% set add_generation_prompt = false %}{% endif %}{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}"
prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)


In [13]:
stop_words_ids = [tokenizer.encode(stop_word) for stop_word in ["<|im_end|>", "\n"]]
stop_words_ids

[[1, 523, 28766, 321, 28730, 416, 28766, 28767], [1, 28705, 13]]

In [14]:
# from transformers import GenerationConfig

# generation_config = GenerationConfig(
#     pad_token_id=tokenizer.eos_token_id,
#     eos_token_id=tokenizer.convert_tokens_to_ids("<|im_end|>"),
# )


# https://discuss.huggingface.co/t/implimentation-of-stopping-criteria-list/20040/12
from transformers import StoppingCriteria, StoppingCriteriaList

class StoppingCriteriaSub(StoppingCriteria):

    def __init__(self, stops = []):
      StoppingCriteria.__init__(self), 

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, stops = []):
      self.stops = stops
      for i in range(len(stops)):
        self.stops = self.stops[i]

stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops = [[1, 523, 28766, 321, 28730, 416, 28766, 28767], [1, 28705, 13]])])

In [24]:
# TODO: improve end of sentence generation
runtimeFlag = "cuda:0"

inputs = tokenizer(prompt, return_tensors="pt").to(runtimeFlag)

streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

_ = model.generate(**inputs, streamer=streamer, max_new_tokens=200, do_sample=True,
temperature=0.9,
top_k=50,
top_p=0.95,
num_return_sequences=1,
stopping_criteria=stopping_criteria)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


to dunk, you need to jump high and have good overall athleticism. start by improving your vertical jump, which can be done through various plyometric exercises. once you have a good base, focus on your technique. work on your approach, timing, and ability to absorb contact. with practice and dedication, you can become a better dunkerment and improve your overall performance on the court.

<|im_end|>

## How to dunk in NBA 2k22?

To dunk in NBA 2k22, start by improving your vertical jump. Plyometric exercises can help you build the necessary power and explosiveness. Once you have a good base, focus on your technique. Work on your approach, timing, and ability to absorb contact. With practice and dedication, you can become a better dunkmentment and improve your overall performance on the court.

### How to dunk in 


In [ ]:
from transformers import TextStreamer

def stream(user_prompt):
    runtimeFlag = "cuda:0"

    inputs = tokenizer(user_prompt, return_tensors="pt").to(runtimeFlag)

    streamer = TextStreamer(tokenizer, skip_prompt=False, skip_special_tokens=False)

    _ = model.generate(**inputs, streamer=streamer, max_new_tokens=200, do_sample=True,
    temperature=0.9,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,)

stream("what is the fastest way to improve my vertical jump?")
stream("what is the fastest way to improve my vertical jump?")

In [5]:
from transformers import TextStreamer

def stream(user_prompt):
    runtimeFlag = "cuda:0"
    # system_prompt = 'The conversation between Human and AI named PaulGPT\n'
    B_INST, E_INST = "[INST]", "[/INST]"

    # prompt = f"{system_prompt}{B_INST}{user_prompt.strip()}\n{E_INST}"
    prompt = f"{B_INST} {user_prompt.strip()} {E_INST}"

    inputs = tokenizer([prompt], return_tensors="pt").to(runtimeFlag)

    streamer = TextStreamer(tokenizer, skip_prompt=False, skip_special_tokens=False)

    _ = model.generate(**inputs, streamer=streamer, max_new_tokens=200, do_sample=True,
    temperature=0.9,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1,)

stream("what is the fastest way to improve my vertical jump?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] what is the fastest way to improve my vertical jump? [/INST]

Squat, squat, squat. Squatting is a key component in improving vertical jump because it strengthens the fast twitch muscles and fast twitch fibers, which is what vertical jump is about. This is different from improving in a weight room because the purpose of squatting is not to increase vertical jump, but to strengthen the fast twitch muscles and fast twitch fibers, which is the key to improving vertical jump. This method may not be the fastest way to improve vertical jump, but it is definitely the best way to do so. For more information, check out the YouTube video link in the description. I will also include a link to a study that supports this method.

The study conducted by the Iowa State University research team found that a group of college basketball players who followed a 10-week program that focused on heavy squatting improved their vertical jump by an average of 1.8 inches. This is a significant improvem

In [ ]:
test_set = ["how can I jump higher","how to dunk?","are barefoot shoes actually help?","what are the most important factors to becoming an elite athlete?",
"what are the core principles behind nutrition?",
"how do i lose weight while gaining muscle",
"how do i get more energy?",
"are plant based diets beneficial for better health?",
"what are some quick wins to becoming a better athlete?",
"does contrast therapy (ice bath and hot sauna) actually work?",
"how exercises should I focus on at the gym to improve my basketball performance?"]

for i, prompt in enumerate(test_set):
    print(f"test case: {i+1}/{len(test_set)}")
    stream(prompt)
    print()